## Gan for double exponential

In [2]:
'''
Generate Dataset. It looks like: 1/100 * exp(1), 100 * exp(1),.....

I transformed the data by applying log transformation and then divided by -15 to make fall into a small range
'''
import numpy as np
dataset = np.zeros(shape=(5000,500))
for j in range(0,5000):
    for i in range(0,500):
        if i % 2 == 1:
            dataset[j][i] = np.log(100 * np.random.exponential()) / -15
        else:
            dataset[j][i] = np.log(1/100 * np.random.exponential()) / -15

### Below is the plot after transforming 

Text(0.5, 1.0, 'Ture distribution after transformation')

In [39]:
import seaborn as sns
sns.distplot(dataset,color='r').set_title("Ture distribution after transformation")
# summarize discriminator performance
# print(epoch, "real acc.",acc_real, "fake acc", acc_fake,'kl divergence',kl_diff)

In [38]:
print("Number of training examples:",5000)
print("Each training example has length of:",500)

Number of training examples: 5000
Each training example has length of: 500


In [20]:
import seaborn as sns, numpy as np
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand,randint
from numpy.random import randn, exponential
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, BatchNormalization
from matplotlib import pyplot
from keras import optimizers
import tensorflow as tf
from numpy.random import random, choice
real_acc=[[],[]]
fake_acc=[[],[]]
g_loss=[[],[]]
d_loss=[[],[]]
# define the standalone discriminator model
def define_discriminator(n_inputs=500):
    model = Sequential()
    model.add(Dense(300, kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(50))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = optimizers.RMSprop(lr=0.0008, decay=1e-10)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=500):
    model = Sequential()
    model.add(Dense(100, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(300))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(300))
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dense(n_outputs))
    model.add(LeakyReLU(alpha=0.1))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = optimizers.Adam(lr=0.0004, decay=1e-10)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# generate n real samples with class labels
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0],n_samples)
    x = dataset[ix]
    y=ones((n_samples,1))
    return x, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n, 1))
    return X, y

# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    sns.set()
    # prepare real samples
    x_real, y_real = generate_real_samples(dataset,n)
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    real_acc[1].append(acc_real)
    real_acc[0].append(epoch)
    fake_acc[1].append(acc_fake)
    fake_acc[0].append(epoch)
    # scatter plot real and fake data points
    sns.distplot(x_real,color='r')
    sns.distplot(x_fake[:,1::2],color='b')
    sns.distplot(x_fake[:,::2],color='y').set_title("Epoch:"+str(epoch))
    file = str(epoch)+'.png'
    pyplot.savefig(file)
    pyplot.close()

def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)
def smooth_negative_labels(y):
    return y + 0.3 + (random(y.shape) * 0.3)


# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=256, n_eval=500):
    # determine half the size of one batch, for updating the discriminator
    bat_per_eop = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        for j in range(bat_per_eop):
            # prepare real samples
            x_real, y_real = generate_real_samples(dataset,half_batch)
            # prepare fake examples
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            y_real = smooth_positive_labels(y_real)
            y_fake = smooth_negative_labels(y_fake)
            # update discriminator
            loss_1=d_model.train_on_batch(x_real, y_real)
            loss_2=d_model.train_on_batch(x_fake, y_fake)
            # prepare points in latent space as input for the generator
            x_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            y_gan = smooth_positive_labels(y_gan)
            # update the generator via the discriminator's error
            loss_3=gan_model.train_on_batch(x_gan, y_gan)
            d_loss.append(loss_1[0]+loss_2[0])
            g_loss.append(loss_3)
            # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)
            d_loss[1].append(loss_1[0]+loss_2[0])
            g_loss[1].append(loss_3)
            d_loss[0].append(i+1)
            g_loss[0].append(i+1)
            print("D loss",loss_1[0]+loss_2[0],"G loss",loss_3)

with tf.device('/device:GPU:3'):

    # size of the latent space
    latent_dim = 100
    # create the discriminator
    discriminator = define_discriminator()
    # create the generator
    generator = define_generator(latent_dim)
    # create the gan
    gan_model = define_gan(generator, discriminator)
    # train model
    train(generator, discriminator, gan_model, latent_dim)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


499 1.0 0.0
D loss 1.243766 G loss 0.45217833


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


999 1.0 0.0
D loss 1.2085061 G loss 0.39968196


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1499 1.0 0.0
D loss 1.2088304 G loss 0.4275194


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1999 1.0 0.0
D loss 1.2230814 G loss 0.42192227


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


2499 1.0 0.0
D loss 1.2298998 G loss 0.43946815


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


2999 1.0 0.0
D loss 1.2326567 G loss 0.42281434


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


3499 1.0 0.0
D loss 1.2277524 G loss 0.4262532


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


3999 1.0 0.0
D loss 1.2536342 G loss 0.4406759


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


4499 1.0 0.019999999552965164
D loss 1.197782 G loss 0.46277928


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


4999 1.0 0.019999999552965164
D loss 1.2040931 G loss 0.473374


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


5499 1.0 0.03999999910593033
D loss 1.1474203 G loss 0.5116054


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


5999 1.0 0.10000000149011612
D loss 1.1138744 G loss 0.5452545


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


6499 0.9900000095367432 0.14000000059604645
D loss 1.1140325 G loss 0.59626603


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


6999 0.9599999785423279 0.2800000011920929
D loss 1.0966914 G loss 0.61887133


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


7499 0.9900000095367432 0.2800000011920929
D loss 1.1133491 G loss 0.618815


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


7999 0.9900000095367432 0.1899999976158142
D loss 1.1403233 G loss 0.6209363


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


8499 1.0 0.25
D loss 1.1750405 G loss 0.63376343


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


8999 0.9900000095367432 0.25999999046325684
D loss 1.1503708 G loss 0.65839785


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


9499 1.0 0.28999999165534973
D loss 1.1722077 G loss 0.6177686


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


9999 0.9900000095367432 0.33000001311302185
D loss 1.0857037 G loss 0.67716694


In [36]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,3))
plt.subplot(121)
plt.title("Real V.S. Fake Accuracy")
line_down, =plt.plot(fake_acc[0],fake_acc[1],color='r',label="fake acc")
line_up, =plt.plot(real_acc[0],real_acc[1],color='b',label="real acc")
plt.legend(handles=[line_up, line_down])
plt.subplot(122)
plt.title("G LOSS V.S. D LOSS")
line_down, =plt.plot(g_loss[0],g_loss[1],color='r',label="G LOSS")
line_up, =plt.plot(d_loss[0],d_loss[1],color='b',label="D LOSS")
plt.legend(handles=[line_up, line_down])
# sns.distplot(dataset[:,::2],color='b')

## Some outputs during training

In [9]:
from IPython.display import Image, HTML, display
from glob import glob
li=['plots/499.png','plots/999.png','plots/1499.png', 'plots/1999.png', 'plots/2499.png', 'plots/2999.png', 'plots/3499.png', 'plots/3999.png', 'plots/4499.png',
   'plots/4999.png', 'plots/5499.png', 'plots/5999.png', 'plots/6499.png', 'plots/6999.png', 'plots/7499.png', 'plots/7999.png', 'plots/8499.png', 'plots/8999.png', 'plots/9499.png',
   'plots/9999.png']
imagesList=''.join( ["<img style='width: 240px; margin: 5px; float: left; border: 0px solid black;' src='%s' />" % str(s) 
                     for s in li ])
display(HTML(imagesList))